<a href="https://colab.research.google.com/github/sidhusmart/Uplimit_Langchain_Course/blob/main/Week1/Langchain_Week1_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries to get it out of the way
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma faiss-gpu pymupdf grandalf gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 903.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 17.8 MB/s eta 0:00

In [ ]:
## Your code here
from langchain.document_loaders import PyMuPDFLoader


pdf_path = "/content/Goldinsurance.pdf"
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()
page_number = 2
num_pages = len(documents)

if 0 <= page_number < num_pages:
    print(f"Content of Page {page_number + 1}:")
    print(documents[page_number].page_content)
    print(f"Metadata of Page {page_number + 1}:")
    print(documents[page_number].metadata)

Content of Page 3:
3
THE AMERICAN EXPRESS® GOLD CARD INSURANCE  
TERMS AND CONDITIONS
American Express® Gold Charge Card Insurance
Policy Number: 09NACGOLD1
Cover is effective from 1 December 2020
Before You Travel…
•	 
This booklet contains important information about Your American 
Express Gold Card insurance. You should read it carefully and keep it 
in a safe place.
•	 
We recommend that You take a copy with You when You travel.  
You can also access a copy online any time by going to  
americanexpress.co.nz/goldinsurance.
•	 
If You need to purchase any essential items because of a delay or lost 
luggage, make sure You use Your American Express Gold Card for all 
purchases.
•	 
If You need to make a claim, keep detailed particulars and proof of 
any loss including all police reports, sales receipts and credit card 
statements showing any purchases made.
•	 
If You have any questions about what is covered and what is not, 
please call 0800 703 702
ACTIVATING YOUR COVER
In orde

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=70
)

all_text = " ".join([doc.page_content for doc in documents])

chunks = text_splitter.split_text(all_text)
print(chunks[10:11])

['AND TRAVEL INSURANCE COVER (SECTION B–F)\nCover is effective for each Trip only when:\n(a) You pay the full fare for a return Trip on Your American Express Gold Card; \nor\n(b) You pay using American Express Membership Rewards points; or\n(c) You use Your Travel Benefit.']


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from google.colab import userdata

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_texts(texts=chunks, embedding=embedding_model)

We have our documents indexed and we can directly start retrieving documents based on a similarity search with a target query.

In [ ]:
search_result = vector_store.similarity_search_with_relevance_scores(query="How much is covered in rental insurance?", k=4)
search_result

[(Document(page_content='Provides cover for Your \nliability for damages for \nInjury to any person or \ndamage or accidental loss \nto property.\nUp to $1,000,000\n•\t\nInjury or loss of \nor damage to \nmaterial property \narising directly \nor indirectly out \nof the ownership, \npossession, \ncontrol or use by \nYou or on Your \nbehalf'),
  0.2729379504489995),
 (Document(page_content='hundred and sixty five (365) \nday period;\n•\t\n$2,500 per event.\n$50 Excess for each and every \nclaim\n•\t\n Eligible Items left \nUnattended in a \nPublic Place\n•\t\nEligible Items left \nin an Unattended \nmotor vehicle\nBUYERS ADVANTAGE COVER\nH\nBuyer’s Advantage Cover\nProvides cover for the'),
  0.21918015093553633),
 (Document(page_content='SUM INSURED\nKEY EXCLUSIONS\nD\nBaggage, Money and \nDocuments\nProvides cover for \ndamaged, destroyed, lost \nor stolen Personal Baggage \nor Money and Documents \nduring Your Trip.\nUp to $10,000 in total in any one \n(1) three hundred and sixty fiv

### Creating the QnA RAG chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

qna_prompt = """
You are an expert credit card specialist who can read and understand the information from credit card information leaflets.
Please use that to answer any questions that are asked.
Please explain in an easy to understand language for average people.
If you can't answer the question, please say so, don't try to make up an answer.

You are provided with the Question and Context below to help you answer the question.

Question: {question}
Context: {context}
Answer:
"""


qna_prompt_template = ChatPromptTemplate.from_messages(
    [
    SystemMessagePromptTemplate.from_template("You are a helpful AI assistant."),
    HumanMessagePromptTemplate.from_template(qna_prompt),
    ]
)

LLM = ChatOpenAI(model_name="gpt-3.5-turbo",
                 temperature=0,
                 openai_api_key=userdata.get('OPENAI_API_KEY'))

retriever = vector_store.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) # only retrive page_content, no need others info

# Chain (2 chains, one chain within the outer chain, the inner chain is retrieval part)
rag_chain = (
    {"context": itemgetter("input") | retriever | format_docs, # another chain CECL format
     # take input from user, pipe to retriever (use vector store) and format
     # this overall from above take care of one elment of RAG chain
     "question": RunnablePassthrough()}
    # below qna_prompt_template gets 2 inputs: context (results from the above pipe), and question (get directly from the original prompt)
    | qna_prompt_template
    | LLM
    | StrOutputParser() # to make sure only take text out
)

# Questions
rag_chain.invoke({"input": "How long does a claim take?"})

"To make a claim under the terms and conditions of the credit card, you should submit your claim within twenty (20) days after the covered loss occurs or as soon as reasonably possible after that. Benefits will be paid once you provide written proof of the legitimate covered loss. You need to provide proof of purchase for the items you are claiming. If you can't provide proof of purchase, your claim may not be paid, and this decision will be made at the discretion of the credit card company. To support your claims, you must provide the original purchase receipt or alternative written proof of the purchase price. Make sure to keep copies of all the documents you send to the Claims Handler. Additionally, disclose all relevant information about the claim, including details of any other insurance cover you may have for the same claim."

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo",
                 temperature=0,
                 openai_api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": itemgetter("input") | retriever | format_docs,
     "question": RunnablePassthrough()}
    | qna_prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
question = "Who should I contact to get an insurance claim?"
rag_chain.invoke({"input": question})

"To get an insurance claim, you should contact Chubb Assistance. It's important to reach out to them as soon as a claim or potential claim arises. You need to contact them before incurring any expenses or as soon as physically possible. Make sure to disclose all relevant information about the claim, including details of any other insurance cover you may have. To support your claim, you will need to provide the original purchase receipt or other proof of the purchase price. Additionally, you must supply all original invoices, receipts, and reports to the Appointed Claims Handler and keep copies of the documents you send. Remember, providing this information is crucial for Chubb Assistance to handle your claim effectively. If you fail to provide the necessary information, they may not be able to assist you with your insurance claim or respond to any complaints or disputes."

In [ ]:
print (rag_chain.get_graph().print_ascii())

           +---------------------------------+        
           | Parallel<context,question>Input |        
           +---------------------------------+        
                    **              ***               
                 ***                   **             
               **                        ***          
       +--------+                           **        
       | Lambda |                            *        
       +--------+                            *        
            *                                *        
            *                                *        
            *                                *        
+----------------------+                     *        
| VectorStoreRetriever |                     *        
+----------------------+                     *        
            *                                *        
            *                                *        
            *                                *        
    +-----

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = "pr-slight-variability-56"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

def predict(message, history):
    return rag_chain.invoke({"input":message})

gr.ChatInterface(predict).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa3d81400c14e5579e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from langsmith import Client

client = Client()

In [ ]:
import uuid
from langsmith import schemas as ls_schemas

dataset_name = f"PillPal QA {str(uuid.uuid4())}"
dataset = client.create_dataset(dataset_name=dataset_name, data_type=ls_schemas.DataType.llm)
for q, a in examples:
    client.create_example(
        inputs={"input": q}, outputs={"output": a}, dataset_id=dataset.id
    )

NameError: name 'examples' is not defined

In [ ]:
from langchain.smith import RunEvalConfig
from langchain_openai import ChatOpenAI

eval_config = RunEvalConfig(
    evaluators=["qa"],
    eval_llm=ChatOpenAI(model_name="gpt-4", openai_api_key=userdata.get('OPENAI_API_KEY'))
)

Let's run the evaluation on the dataset. You get the links to where you can track the run and at the end the output is also printed here.

In [ ]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=lambda: rag_chain,
    evaluation=eval_config,
)

View the evaluation results for project 'stupendous-music-80' at:
https://smith.langchain.com/o/65a89521-d351-585d-80e0-b5c278d2180b/datasets/c1126ec2-01d9-4d3b-93d5-34e4c515b5a0/compare?selectedSessions=39f6f667-2b27-4fbb-8d96-d66880ae76cc

View all tests for Dataset PillPal QA 6610cee9-070c-43c9-b1db-bede57fe8651 at:
https://smith.langchain.com/o/65a89521-d351-585d-80e0-b5c278d2180b/datasets/c1126ec2-01d9-4d3b-93d5-34e4c515b5a0
[------------------------------------------------->] 10/10

{'project_name': 'stupendous-music-80',
 'results': {'f38cd15c-282e-43cf-8fda-f388aeff83f2': {'input': {'input': 'What should I do if I miss a dose?'},
   'feedback': [EvaluationResult(key='correctness', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('037d4c11-bd78-43c1-ac34-50dfce454165'))}, feedback_config=None, source_run_id=None, target_run_id=None)],
   'execution_time': 2.173642,
   'run_id': 'dee089a2-dffa-4ac7-b719-638e81f7dd0f',
   'output': "If you forget to take Cetirizine Hydrochloride Tablets, do not take a double dose to make up for the missed dose. Instead, take the missed tablet as soon as you remember, but wait at least 24 hours before taking your next tablet. It's important to follow this guidance to avoid any potential risks or side effects associated with taking too much of the medication. If you have any concerns or questions about missed doses or your medication regimen, it's best to consult your doctor o